# Question检索 -- 丰富短句信息

---

In [8]:
import os
import re
from tqdm import tqdm
import traceback
import random
import sys
import pprint
import jieba

sys.path.insert(0, "/home/team55/notespace/zengbin")

from jddc.config import PreConfig
from jddc.utils import write_file, read_file, save_to_pkl, read_from_pkl, create_logger
from jddc.obj import Session, Sentence
from jddc.seg import JiebaSeg
from jddc.similarities import SentenceSimilarity

conf = PreConfig()
logger = create_logger(name='pre', log_file=conf.log_file, cmd=conf.cmd_log)

In [2]:
def load_sessions():
    pkl_sessions = conf.pkl_sessions
    print("load sessions from %s" % pkl_sessions)
    sess_objs = read_from_pkl(pkl_sessions)
    return sess_objs

In [3]:
sessions = load_sessions()

load sessions from /submitwork/data/temp/all_sessions.pkl


## 获取全部questions
---

In [4]:
questions = []
for sess in tqdm(sessions, ncols=100, desc="get questions"):
    qs = " ".join([x[1].replace("\t", " ") for x in sess.qas_merged if x[0]=="0"])
    if 50 < len(qs) < 200:
        questions.append(qs)

get questions: 100%|██████████████████████████████████| 1025140/1025140 [00:05<00:00, 178500.90it/s]


In [5]:
len(questions)

730713

## 构建检索模型
---

In [6]:
ss_model = os.path.join(conf.temp_path, "q_retrieve_100000.pkl")

In [7]:
q_retrieve = read_from_pkl(ss_model)

In [ ]:
seg = JiebaSeg(conf.file_stopwords)
q_retrieve = SentenceSimilarity(seg)
q_retrieve.set_sentences(questions[:100000])

In [ ]:
q_retrieve.tfidf_model()
q_retrieve.clear()
save_to_pkl(file=ss_model, data=q_retrieve)

## 测试检索模型

---

In [42]:
i = random.choice(range(100000, len(questions)))

In [43]:
for turn in range(1, 6):
    print("第%s轮" % str(turn))
    print("-" * 88)
    print(" ".join(questions[i].split(" ")[:turn]))
    res = q_retrieve.similarity(" ".join(questions[i].split(" ")[:turn]), top=5)
    pprint.pprint([x for x in res if x[1]>0.2])
    print("#" * 88)

第1轮
----------------------------------------------------------------------------------------
亲
[(91643,
  0.87640554,
  '您好亲 亲 我们后台客户申请退货，但是顾客又不想退了，我们这边为什么取消不了呢 刚刚客服说终止取件  我们具体怎么操作呢 亲 可以的 亲 '
  '单号的话是顾客收货的单号吗 亲 订单号可以吗 [数字x] 亲 这是订单号 亲 好的 那这样我们告知客户了  这个单子就可以的吗 好的 好的亲 后台呢亲 '
  '一直挂着怎么办呢 好的亲 好的亲 谢谢哦 没有了哦 好的亲'),
 (67922,
  0.8344149,
  '在的亲 在的亲 [数字x] 亲这个信息的电话能帮忙改下? [数字x] 亲帮忙改下 这样配送的时候 快递就直接联系新的了吧 没了亲'),
 (94907,
  0.80604154,
  '亲，在? 亲，在? 亲，在? 亲，如果我们京东的客服要求开发票，我们可以额外增加税点? 亲，是的 亲，意思是页面选择了可以开票就不能收取 '
  '那要是显示不可以呢? 亲，好的'),
 (93352,
  0.7826351,
  '开通京东物流 我是商家 亲 开通京东配送  亲 怎么联系 亲 不知道的呢  亲 您不知道吗  亲亲 哦 好的  谢谢 没有 亲'),
 (5430,
  0.7571201,
  '亲 [订单编号:[ORDERID_10394268]，订单金额:[金额x]，下单时间:[日期x] [时间x]] 亲 亲 麻烦同意下我的退货申请 是的 '
  '好 好的 好的 谢谢')]
########################################################################################
第2轮
----------------------------------------------------------------------------------------
亲 电视机我们这边收到了
[(66130,
  0.80662376,
  '我想看一下我的电视机什么时候发货什么时候到 我想看一下我的电视机什么时候发货什么时候 在吗? 我